<a href="https://colab.research.google.com/github/18187558737/ResNet8_CIFAR10_PGD_2/blob/main/ResNet18_CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
from google.colab import drive
drive.mount('/content/drive') # 授权 Google Drive
sys.path.append('/content/drive/MyDrive/320') # 此为你的自定义模块的存储路径

Mounted at /content/drive


In [2]:

from ResNet import *

from load_local_cifar10 import *
from easydict import EasyDict
from projected_gradient_descent import projected_gradient_descent
from fast_gradient_method import fast_gradient_method

In [3]:
cifar10_dir = '/content/drive/MyDrive/cifar-10-batches-py'
(x_train, y_train), (x_test, y_test) = load_data(cifar10_dir)
x_train, x_test = x_train / 255.0, x_test / 255.0

In [4]:
model = ResNet18([2, 2, 2, 2])
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['sparse_categorical_accuracy'])



checkpoint_save_path = "/content/drive/MyDrive/320/ResNet50.ckpt"
if os.path.exists(checkpoint_save_path + '.index'):
    print('-------------load the model-----------------')
    model.load_weights(checkpoint_save_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_save_path,
                             save_weights_only=True,
                             save_best_only=True)

history = model.fit(x_train, y_train, batch_size=128, epochs=5, validation_data=(x_test, y_test), validation_freq=1,
                    callbacks=[cp_callback])
model.summary()

-------------load the model-----------------
Epoch 1/5
391/391 [==============================] - 118s 257ms/step - loss: 0.2770 - sparse_categorical_accuracy: 0.9121 - val_loss: 0.6983 - val_sparse_categorical_accuracy: 0.8011
Epoch 2/5
391/391 [==============================] - 92s 236ms/step - loss: 0.2050 - sparse_categorical_accuracy: 0.9383 - val_loss: 1.1674 - val_sparse_categorical_accuracy: 0.7230
Epoch 3/5
391/391 [==============================] - 92s 236ms/step - loss: 0.1522 - sparse_categorical_accuracy: 0.9581 - val_loss: 0.9271 - val_sparse_categorical_accuracy: 0.7642
Epoch 4/5
391/391 [==============================] - 93s 237ms/step - loss: 0.1219 - sparse_categorical_accuracy: 0.9669 - val_loss: 0.8715 - val_sparse_categorical_accuracy: 0.7696
Epoch 5/5
391/391 [==============================] - 92s 236ms/step - loss: 0.0999 - sparse_categorical_accuracy: 0.9749 - val_loss: 1.0083 - val_sparse_categorical_accuracy: 0.7645
Model: "res_net18"
_________________________

In [5]:
test_acc_clean = tf.metrics.SparseCategoricalAccuracy()
test_acc_fgsm = tf.metrics.SparseCategoricalAccuracy()
test_acc_pgd = tf.metrics.SparseCategoricalAccuracy()
eps = 0.05

In [6]:
cifar10_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
cifar10_test = tf.data.Dataset.from_tensor_slices((x_test[0:1000], y_test[0:1000]))
data = EasyDict(train=cifar10_train, test=cifar10_test)

In [7]:
progress_bar_test = tf.keras.utils.Progbar(1000)

for x, y in cifar10_test:
    x = np.expand_dims(x, axis=0)
    y_pred = model(x)
    test_acc_clean(y, y_pred)

    x_fgm = fast_gradient_method(model, x, eps, np.inf)
    y_pred_fgm = model(x_fgm)
    test_acc_fgsm(y, y_pred_fgm)

    x_pgd = projected_gradient_descent(model, x, eps, 0.01, 40, np.inf)
    y_pred_pgd = model(x_pgd)
    test_acc_pgd(y, y_pred_pgd)

    progress_bar_test.add(x.shape[0])
  
print(
    "test acc on clean examples (%): {:.3f}".format(test_acc_clean.result() * 100)
)                                                                                                             
print(
    "test acc on FGM adversarial examples (%): {:.3f}".format(
        test_acc_fgsm.result() * 100
    )
)
print(
    "test acc on PGD adversarial examples (%): {:.3f}".format(
        test_acc_pgd.result() * 100
    ))

1000/1000 [==============================] - 885s 882ms/step
test acc on clean examples (%): 77.900
test acc on FGM adversarial examples (%): 11.600
test acc on PGD adversarial examples (%): 11.300


In [11]:
# CIFAR - 10

# To decode the files
import pickle
# For array manipulations
import numpy as np
# To make one-hot vectors
from tensorflow.python.keras.utils import np_utils
# To plot graphs and display images
from matplotlib import pyplot as plt

# constants

path = "/content/drive/MyDrive/cifar-10-batches-py/"  # Path to data

# Height or width of the images (32 x 32)
size = 32

# 3 channels: Red, Green, Blue (RGB)
channels = 3

# Number of classes
num_classes = 10

# Each file contains 10000 images
image_batch = 10000

# 5 training files
num_files_train = 5

# Total number of training images
images_train = image_batch * num_files_train


# https://www.cs.toronto.edu/~kriz/cifar.html


def unpickle(file):
    # Convert byte stream to object
    with open(path + file, 'rb') as fo:
        print("Decoding file: %s" % (path + file))
        dict = pickle.load(fo, encoding='bytes')

    # Dictionary with images and labels
    return dict


def convert_images(raw_images):
    # Convert images to numpy arrays

    # Convert raw images to numpy array and normalize it
    raw = np.array(raw_images, dtype=float) / 255.0

    # Reshape to 4-dimensions - [image_number, channel, height, width]
    images = raw.reshape([-1, channels, size, size])

    images = images.transpose([0, 2, 3, 1])

    # 4D array - [image_number, height, width, channel]
    return images


def load_data(file):
    # Load file, unpickle it and return images with their labels

    data = unpickle(file)

    # Get raw images
    images_array = data[b'data']

    # Convert image
    images = convert_images(images_array)
    # Convert class number to numpy array
    labels = np.array(data[b'labels'])

    # Images and labels in np array form
    return images, labels


def get_test_data():
    # Load all test data

    images, labels = load_data(file="test_batch")

    # Images, their labels and
    # corresponding one-hot vectors in form of np arrays
    return images, labels, np_utils.to_categorical(labels, num_classes)


def get_train_data():
    # Load all training data in 5 files

    # Pre-allocate arrays
    images = np.zeros(shape=[images_train, size, size, channels], dtype=float)
    labels = np.zeros(shape=[images_train], dtype=int)

    # Starting index of training dataset
    start = 0

    # For all 5 files
    for i in range(num_files_train):
        # Load images and labels
        images_batch, labels_batch = load_data(file="data_batch_" + str(i + 1))

        # Calculate end index for current batch
        end = start + image_batch

        # Store data to corresponding arrays
        images[start:end, :] = images_batch
        labels[start:end] = labels_batch

        # Update starting index of next batch
        start = end

    # Images, their labels and
    # corresponding one-hot vectors in form of np arrays
    return images, labels, np_utils.to_categorical(labels, num_classes)


def get_class_names():
    # Load class names
    raw = unpickle("batches.meta")[b'label_names']

    # Convert from binary strings
    names = [x.decode('utf-8') for x in raw]

    # Class names
    return names


def plot_images(images, labels_true, class_names, labels_pred=None):
    assert len(images) == len(labels_true)

    # Create a figure with sub-plots
    fig, axes = plt.subplots(3, 3, figsize=(8, 8))

    # Adjust the vertical spacing
    if labels_pred is None:
        hspace = 0.2
    else:
        hspace = 0.5
    fig.subplots_adjust(hspace=hspace, wspace=0.3)

    for i, ax in enumerate(axes.flat):
        # Fix crash when less than 9 images
        if i < len(images):
            # Plot the image
            ax.imshow(images[i], interpolation='spline16')

            # Name of the true class
            labels_true_name = class_names[labels_true[i]]

            # Show true and predicted classes
            if labels_pred is None:
                xlabel = "True: " + labels_true_name
            else:
                # Name of the predicted class
                labels_pred_name = class_names[labels_pred[i]]

                xlabel = "True: " + labels_true_name + "\nPredicted: " + labels_pred_name

            # Show the class on the x-axis
            ax.set_xlabel(xlabel)

        # Remove ticks from the plot
        ax.set_xticks([])
        ax.set_yticks([])

    # Show the plot
    plt.show()


def plot_model(model_details):
    # Create sub-plots
    fig, axs = plt.subplots(1, 2, figsize=(15, 5))

    # Summarize history for accuracy
    axs[0].plot(range(1, len(model_details.history['acc']) + 1), model_details.history['acc'])
    axs[0].plot(range(1, len(model_details.history['val_acc']) + 1), model_details.history['val_acc'])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1, len(model_details.history['acc']) + 1), len(model_details.history['acc']) / 10)
    axs[0].legend(['train', 'val'], loc='best')

    # Summarize history for loss
    axs[1].plot(range(1, len(model_details.history['loss']) + 1), model_details.history['loss'])
    axs[1].plot(range(1, len(model_details.history['val_loss']) + 1), model_details.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1, len(model_details.history['loss']) + 1), len(model_details.history['loss']) / 10)
    axs[1].legend(['train', 'val'], loc='best')

    # Show the plot
    plt.show()


def visualize_errors(images_test, labels_test, class_names, labels_pred, correct):
    incorrect = (correct == False)

    # Images of the test-set that have been incorrectly classified.
    images_error = images_test[incorrect]

    # Get predicted classes for those images
    labels_error = labels_pred[incorrect]

    # Get true classes for those images
    labels_true = labels_test[incorrect]

    # Plot the first 9 images.
    plot_images(images=images_error[0:9],
                labels_true=labels_true[0:9],
                class_names=class_names,
                labels_pred=labels_error[0:9])


def predict_classes(model, images_test, labels_test):
    # Predict class of image using model
    class_pred = model.predict(images_test, batch_size=32)

    # Convert vector to a label
    labels_pred = np.argmax(class_pred, axis=1)

    # Boolean array that tell if predicted label is the true label
    correct = (labels_pred == labels_test)

    # Array which tells if the prediction is correct or not
    # And predicted labels
    return correct, labels_pred



In [12]:
class_names = get_class_names()
print(class_names)

num_classes = len(class_names)
print(num_classes)

# Hight and width of the images
IMAGE_SIZE = 32
# 3 channels, Red, Green and Blue
CHANNELS = 3

images_train, labels_train, class_train = get_train_data()
print(labels_train)

print(class_train)

images_test, labels_test, class_test = get_test_data()
print("Training set size:\t", len(images_train))
print("Testing set size:\t", len(images_test))

scores = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))


y_pred = model.predict(images_test, batch_size=32)
print(y_pred[0])

y_pred = np.argmax(y_pred, axis=1)
print(y_pred)

correct = (y_pred == labels_test)
print(correct)


print("Number of correct predictions: %d" % sum(correct))

num_images = len(correct)
print("Accuracy: %.2f%%" % ((sum(correct)*100)/num_images))

print("Number of incorrect predictions: %d" % (num_images-sum(correct)))
incorrect = (correct == False)

# Images of the test-set that have been incorrectly classified.
images_error = x_test[incorrect]

# Get predicted classes for those images
labels_error = y_pred[incorrect]

# Get true classes for those images
labels_true = y_test[incorrect]
num_error_class = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
for x in labels_true:
  if x == 0:
    num_error_class[0] += 1
  if x == 1:
    num_error_class[1] += 1
  if x == 2:
    num_error_class[2] += 1
  if x == 3:
    num_error_class[3] += 1
  if x == 4:
    num_error_class[4] += 1
  if x == 5:
    num_error_class[5] += 1
  if x == 6:
    num_error_class[6] += 1
  if x == 7:
    num_error_class[7] += 1
  if x == 8:
    num_error_class[8] += 1
  if x == 9:
    num_error_class[9] += 1
print(num_error_class)

Decoding file: /content/drive/MyDrive/cifar-10-batches-py/batches.meta
['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
10
Decoding file: /content/drive/MyDrive/cifar-10-batches-py/data_batch_1
Decoding file: /content/drive/MyDrive/cifar-10-batches-py/data_batch_2
Decoding file: /content/drive/MyDrive/cifar-10-batches-py/data_batch_3
Decoding file: /content/drive/MyDrive/cifar-10-batches-py/data_batch_4
Decoding file: /content/drive/MyDrive/cifar-10-batches-py/data_batch_5
[6 9 9 ... 9 1 1]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]
Decoding file: /content/drive/MyDrive/cifar-10-batches-py/test_batch
Training set size:	 50000
Testing set size:	 10000
Accuracy: 76.45%
[8.2540224e-05 4.4539389e-03 4.4646654e-06 9.9091643e-01 1.3546454e-04
 3.9144475e-03 4.2013451e-04 4.5082499e-05 1.8119079e-05 9.3349909e-06]
[3 8 1 ... 5 1 7]
[ True  